# DEEPWALK RECOMMENDER

- Hoćemo da primenimo deepwalk algoritam na sistem za preporučivanje. Konkretno, koristićemo skup movie-lense 100k.
- DeepWalk je pristup za učenje reprezentacija čvorova u mrežama. Naučene reprezentacije čvorova su vektori u neprekidnom vektorskom prostoru.
- DeepWalk koristi kratke slučajne šetnje kroz graf, za svaki čvor, tretirajući šetnje kao ulazne rečenice za word2vec, kako bi dobio latentnu reprezentaciju čvorova.

<img src='assets/embedding.jpg'>

- Koraci:
   - Kreiranje grafa na osnovu movie lense skupa podataka
   - Formiranje korpusa - skupa slučajnih šetnji kroz graf, za svaki čvor (moguće je krenuti više puta iz istog čvora)
   - Primena word2vec na korpus u cilju dobijanja vektorskih reprezentacija svakog čvora
   - Testiranje dobijenog modela na test skupu, korišćenjem k najblizih suseda u dobijenoj reprezentaciji čvorova

In [ ]:
import random
from tqdm.notebook import tqdm
from gensim.models import Word2Vec
from sklearn import metrics
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise import model_selection
from surprise import accuracy
from surprise import Reader
from surprise.prediction_algorithms.predictions import Prediction

Definišemo dve pomoćne funkcije koje pretvaraju n-torku u string i obrnuto, zbog upotrebe word2vec koji prihvata listu rečenica.

In [61]:
def stringToTuple(s):
    strings=s.split("_")
    if len(strings)==2:
        return (strings[0],int(strings[1]))
    else:
        return (strings[0],int(strings[1]),float(strings[2]))

def tupleToString(t):
    if len(t)==2:
        return str(t[0])+"_"+str(t[1])
    else:
        return str(t[0])+"_"+str(t[1])+"_"+str(t[2])

## Klasa kojom je predstavljen sistem za preporučivanje

- Klasa sadrži potrebne metode za kreiranje i upotrebu sistema za preporučivanje korišćenjem DeepWalk algoritma. 
- Srž je funkcija **fit**, koja kreira Word2Vec model.
- Ideja da reprezentacije učimo na osnovu verovatnoća istovremenih pojavljivanja čvorova. Za čvor $v_i$ procenjujemo verovatnoće pojavljivanja ostalih čvorova grafa unutra njegovog konteksta $\{v_{i-w}, \dots, v_{i-1}, v_{i+1}, \dots v_{i+w}\}$ prilikom slučajne šetnje.
- S obzirom da radimo sa grafom koji ima skup čvorova $V$, hoćemo da nađemo funkciju 
$\Phi: V \mapsto R^{|V|\times d}$, koja svakom čvoru dodelju reprezentaciju u prostoru odgovarajuće dimenzije ($d$ je metaparametar). Drugim rečima, dobijamo matricu dimenzije 
$R^{|V|\times d}$ u kojoj redovi predstavljaju reprezentacije odgovarajućih čvorova.
- Ono što Word2Vec radi je upravo to, za svaku slučajnu šetnju preračunava matricu $\Phi$ koristeći neuronsku mrežu sa jednim skrivenim slojem veličine $d$. 
<img src='assets/word2vec-net.png'>

In [1]:
class DeepWalkRecommender():
    """
    Parametri:
    w: dužina prozora
    epochs: broj epoha
    t: dužina slučajnih šetnji
    latent_dim: dimenzija izlaznog prostora atributa
    k: broj najbližih suseda u izlaznom prostoru atributa
    """
    def __init__(self, w=8, epochs = 10, t = 30,latent_dim=32,k=50):        
        self.w = w
        self.epochs = epochs
        self.t = t
        self.latent_dim = latent_dim
        self.k=k
        
    def __build_graph__(self, data):
        """
        Funkcija koja kreira graf iz skupa podataka. 
        Čvorovi grafa su oblika (oznaka,redniBroj[, ocena]),
        pri čemu je oznaka 'u' za korisnika, 'm' za film,
        redniBroj je identifikator filma/korisnika,
        a ocena predstavlja ocenu kojom je ocenjen film
        Ukoliko je korisnik 'u' ocenio film 'm', onda dodajemo granu izmedju 'u' i 'm'
        """
        G = nx.Graph()
        
        for user, movie, rating in data.all_ratings():
            user_node = ('u', int(data.to_raw_uid(user)))
            movie_node = ('m', int(data.to_raw_iid(movie)), rating)
            G.add_edge(user_node, movie_node)
        
        return G
    
    def __build_val_map__(self, data):
        return {(int(user), int(movie)) : float(rating) for (user, movie, rating) in data}
    
    def make_corpus(self,G):
        """
        Funkcija koja kreira korpus koristeći graf G. 
        Korpus je lista slučajnih šetnji sa početkom u čvorovima grafa, moguće više puta,
        u zavisnosti od parametra epochs
        """
        corpus=[]
        nodes=list(G.nodes)
        for _ in tqdm(range(self.epochs),total=self.epochs):
            np.random.shuffle(nodes)
            for node in nodes:
                walk = [node]
                for i in range(1,self.t):
                    if len(list(G.neighbors(walk[-1]))) == 0:
                        break
                    walk.append(random.choice(list(G.neighbors(walk[-1]))))
                corpus.append([tupleToString(word) for word in walk])    
        return corpus
    
    def fit(self, data):
        """
        Funkcija koja predstavlja sam DeepWalk algoritam. Funkcija kreira word2vec model,
        koristeći prethodne funkcije i čuva ga u 
        self.word2vec za kasniju upotrebu.
        """
        self.possible_ratings = sorted(set([rating for (_, _, rating) in data.all_ratings()]))
        self.mean_rating = data.global_mean
        
        G = self.__build_graph__(data)
        self.G_train=G
        
        corpus=self.make_corpus(G)
        
        self.word2vec=Word2Vec(corpus, size = self.latent_dim, window = self.w, min_count = 0, sg = 1, hs = 1, workers = 4)
        return self.word2vec
        
    def test(self,test_data):
        """
        Funkcija koja predviđa ocene za svaki par korisnik - film u test skupu.
        Dakle, na osnovu modela dobijenog na skupu za treniranje,
        nalazi k najbližih suseda, računa njihovu prosečnu ocenu za odgovarajući film
        i nju prijavljuje kao predviđanje. 
        U slučaju da od k najbližih suseda niko nije ocenio odgovarajući film,
        prijavljuje se ukupna prosečna ocena
        """
        predictions=[]
        for user,movie,rating in test_data:
            user=int(user)
            movie=int(movie)
            
            similar_users=self.word2vec.wv.most_similar(positive = [tupleToString(('u',user))], topn = self.k)
            
            rating_sum=0
            rating_cnt=0
            for similar_user in similar_users:
                for possible_rating in self.possible_ratings:
                    if ('m',movie,possible_rating) in self.G_train.neighbors(stringToTuple(similar_user[0])):
                        rating_sum+=possible_rating
                        rating_cnt+=1
            
            if rating_cnt>0:
                predicted_rating=rating_sum/rating_cnt
            else:
                predicted_rating = self.mean_rating
            details={'s':True}
            predictions.append(Prediction(uid = user,
                                          iid = movie,
                                          r_ui = rating,
                                          est = float(predicted_rating),
                                          details = details))
        return predictions

    def gridSearch(self,train_data,val_data):
        """
        Pomoćna funkcija koja pronalazi optimalne vrednosti parametara,
        pomoću skupa za validaciju
        """
        ws=[16,8,4,2]
        ts=[30,20,10]
        ks=[50,25]
        epochs=[20,10]
        dimensions=[32,16,8,4,2]
        best_params={}
        best_rmse=1000
        out=open('gridSearch.txt','a')
        i=1
        iterations=len(ws)*len(ts)*len(ks)*len(epochs)*len(dimensions)
        
        for w in ws:
            for t in ts:
                for epoch in epochs:
                    for dim in dimensions:
                        for k in ks:
                            self.w=w
                            self.k=k
                            self.latent_dim=dim
                            self.epochs=epoch
                            self.t=t
                            
                            s='Korak: '+str(i)+'/'+str(iterations)+'\n'+'w='+str(w)+'\n'+'t='+str(t)+'\n'+'epochs='+str(epoch)+'\n'+'dim='+str(dim)+'\n'+'k='+str(k)+'\n'
                            out.write(s)
                            out.flush()
                            
                            self.fit(train_data)
                            predictions=self.test(val_data)
                            rmse=accuracy.rmse(predictions)
                            
                            out.write('RMSE='+str(rmse)+'\n')
                            out.write('------------------------\n')
                            out.flush()
                            i+=1
                            
                            if rmse<best_rmse:
                                best_rmse=rmse
                                best_params['w']=w
                                best_params['t']=t
                                best_params['epochs']=epoch
                                best_params['k']=k
                                best_params['dim']=dim
        out.close()                        
        return best_params,best_rmse

In [56]:
d=DeepWalkRecommender()

data = Dataset.load_builtin('ml-100k')
train_data, test_data = model_selection.train_test_split(data, test_size = 0.3,random_state=7)
val_data=test_data[:15000]

best_params,best_rmse=d.gridSearch(train_data,val_data)
best_params

Korak: 1 / 240



RMSE: 1.1281
Korak: 2 / 240



RMSE: 1.1492
Korak: 3 / 240



RMSE: 1.1121
Korak: 4 / 240



RMSE: 1.1424
Korak: 5 / 240


...


RMSE: 1.1607
Korak: 238 / 240



RMSE: 1.1778
Korak: 239 / 240



RMSE: 1.2271
Korak: 240 / 240



RMSE: 1.2155


{'w': 2, 't': 10, 'epochs': 10, 'k': 50, 'dim': 8}

In [64]:
train_data1, test_data1 = model_selection.train_test_split(data, test_size = 0.15,random_state=7)

d_final=DeepWalkRecommender(w=best_params['w'],
                            epochs=best_params['epochs'],
                            t=best_params['t'],
                            latent_dim=best_params['dim'],
                            k=best_params['k'])

#test_data1 je isto kao test_data[15000:], provereno                            
model=d_final.fit(train_data1)
predictions=d_final.test(test_data1)
print("FINAL:")
final_rmse=accuracy.rmse(predictions)


FINAL:
RMSE: 1.0657


In [57]:
best_rmse

1.0733986399250022

Možemo uporediti naš sistem sa bibliotečkim SVD:

In [59]:
svd = SVD(n_epochs = 10, lr_all = 0.005, reg_all = 0.4)
svd.fit(train_data1)
predictionsSVD = svd.test(test_data1)
accuracy.rmse(predictionsSVD)
accuracy.mae(predictionsSVD)

RMSE: 0.9622
MAE:  0.7709


0.7709396085417138

In [15]:
model.save('deepwalk.model')